In [7]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

# ChromeDriver auto install method
from webdriver_manager.chrome import ChromeDriverManager
# for not defined By
from selenium.webdriver.common.by import By

executable_path = ChromeDriverManager().install()
source_url = "https://www.mangoplate.com/search/%EA%B0%95%EB%82%A8%EC%97%AD"

driver = webdriver.Chrome(executable_path)
driver.get(source_url)

req = driver.page_source
soup = BeautifulSoup(req, "html.parser")
info_list = soup.find_all(name="div", attrs={"class":"info"})

C:\Users\yelin\AppData\Local\Temp\ipykernel_18476\427093990.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path)


In [8]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = []
page_url_base = "https://www.mangoplate.com"
for index in range(0, len(info_list)):
    info = info_list[index]
    review_url = info.find(name="a")
    if review_url is not None:
        page_urls.append(page_url_base + review_url.get("href"))

# 중복 url을 제거합니다.
page_urls = list(set(page_urls))

# 다섯 개의 페이지를 출력합니다.
for page in page_urls[:5]:
    print(page)

# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://www.mangoplate.com/restaurants/1lddS4GuHX
https://www.mangoplate.com/restaurants/cxUdZBLDuCiI
https://www.mangoplate.com/restaurants/0Vq0bf1rZJLZ
https://www.mangoplate.com/restaurants/pWAvmqr2i7TE
https://www.mangoplate.com/restaurants/51t4XhhxEGpT


In [22]:
columns = ['score', 'review']
df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(mac_path)  # for Mac
# driver = webdriver.Chrome(window_path)  # for Windows
for page_url in page_urls:
    
    # 상세보기 페이지에 접속합니다
    driver.get(page_url)
    
    # 리뷰를 크롤링합니다
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_list = soup.find(name="section", attrs={"class":"RestaurantReviewList"})
    
    # 평가 정보('맛있다' 마크)를 가져옵니다.
    review_recommend_list = review_list.find_all(name="div", attrs={"class":"RestaurantReviewItem__Rating RestaurantReviewItem__Rating--Recommend"})
    
    # 리뷰를 가져옵니다.
    review_text_list = review_list.find_all(name="div", attrs={"class":"RestaurantReviewItem__ReviewContent"})
    
    for review_recommend, review_text in zip(review_recommend_list, review_text_list):
        row = [review_recommend.find(name="span").text, review_text.find(name="p").text]
        series = pd.Series(row, index=df.columns)
        df = pd.concat(series)
        
driver.close()

C:\Users\yelin\AppData\Local\Temp\ipykernel_18476\236634335.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(mac_path)  # for Mac


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "Series"

In [19]:
df.head()

,score,review
0,맛있다,"\n 한줄평: 숙성한우집이지만, 점심에만 파는 김치찌개전골이 은근 ..."
1,맛있다,"\n 컨디먼츠 필요없는, 두 달 드라이에이징 암소한우 티본 스테이크..."
2,맛있다,\n 금액은 조금 있지만 조용한 분위기에서 맛있게 먹을 수 있어서 ...
3,맛있다,\n 한줄평: 강남역에서 건조숙성한우로 유명한 맛집. 점심 불고기정...
4,맛있다,"\n 특오뎅탕 25,000원특철판치즈 25,000원어묵탕을 좋아하는..."
